In [ ]:
import pandas as pd

print("Tout fonctionne, Pandas est prêt !")

In [ ]:
# Correction du chemin :
# 1. On enlève le nom du dossier parent (car on est déjà dedans)
# 2. On met des slashs normaux /
# 3. On corrige la fin du fichier (.gz au lieu de .gzip)

chemin = "données/title.basics.tsv.gz"

# On charge le fichier
df_basics = pd.read_csv(chemin, sep='\t', compression='gzip', nrows=50)

df_basics.head()

In [ ]:
# On charge tout le fichier (j'ai enlevé nrows)
# low_memory=False permet d'éviter un avertissement technique sur les types de colonnes mélangés
df_basics = pd.read_csv(chemin, sep='\t', compression='gzip', low_memory=False)

print("Chargement terminé ! Le fichier est en mémoire.")

In [ ]:
# Affiche le résumé des données : nombre de lignes, noms des colonnes, types de données
df_basics.info()

In [ ]:
# Compte combien il y a de films, de courts-métrages, d'épisodes, etc.
df_basics['titleType'].value_counts()

In [ ]:
# 1. LE FILTRAGE : On ne garde que les lignes où titleType est 'movie'
# .copy() permet de créer un tableau indépendant pour éviter des avertissements bizarres plus tard
df_movies = df_basics[df_basics['titleType'] == 'movie'].copy()

print(f"Après filtrage, il nous reste {len(df_movies)} films. On respire !")

# 2. LE NETTOYAGE : On convertit les colonnes en nombres
# 'coerce' veut dire : "  \N, transforme-le en NaN (vide)"
cols_to_fix = ['startYear', 'runtimeMinutes']

for col in cols_to_fix:
    df_movies[col] = pd.to_numeric(df_movies[col], errors='coerce')

# 3. VERIFICATION
print("\nNouveaux types de données :")
df_movies.info()

In [ ]:
# 1. On groupe par année et on calcule la moyenne de la durée
# On filtre les années > 1900 pour éviter les erreurs de saisie historiques bizarres
duree_par_annee = df_movies[df_movies['startYear'] >= 1900].groupby('startYear')['runtimeMinutes'].mean()

# 2. On fait un graphique simple
# (Pandas utilise une librairie appelée Matplotlib intégrée pour ça)
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6)) # On définit la taille de l'image
duree_par_annee.plot()      # On trace la courbe

plt.title("Évolution de la durée moyenne des films (1900-2023)")
plt.xlabel("Année")
plt.ylabel("Durée moyenne (minutes)")
plt.grid(True) # Ajoute une grille pour mieux lire
plt.show()

In [ ]:
%pip install matplotlib

In [ ]:
# 1. Charger le fichier des notes (Ratings)
# On vérifie le chemin (c'est souvent title.ratings.tsv.gz)
chemin_ratings = "données/title.ratings.tsv.gz"
df_ratings = pd.read_csv(chemin_ratings, sep='\t', compression='gzip')

print(f"Nous avons chargé {len(df_ratings)} notes de films.")

# 2. LA FUSION MAGIQUE (Merge) 🤝
# On a deux tableaux : df_movies (les infos) et df_ratings (les notes)
# Ils ont un point commun : la colonne 'tconst' (l'identifiant unique du film, ex: tt000001)
# On va les coller ensemble grâce à cette colonne.

df_complet = pd.merge(df_movies, df_ratings, on='tconst')

print(f"Fusion réussie ! Nous avons maintenant un super-tableau de {len(df_complet)} films avec leurs notes.")

# 3. Petit aperçu du résultat
df_complet.head()

In [ ]:
# 1. FILTRE DE POPULARITÉ (Fiabilité)
# On ne garde que les films ayant au moins 100 votes.
# Tu pourras changer ce chiffre plus tard si tu veux être plus sélective.
df_fiable = df_complet[df_complet['numVotes'] >= 100].copy()

# 2. FILTRE DE GENRE (Ciblage Art et Essai)
# On cherche les films dont le genre contient "Drama", "Biography" ou "Documentary"
# Le symbole | veut dire "OU" en langage informatique regex
mots_cles = 'Drama|Biography|Documentary|History'

# On garde uniquement ceux qui correspondent
df_art_essai = df_fiable[df_fiable['genres'].str.contains(mots_cles, na=False)]

print(f"Base finale pour Tours : {len(df_art_essai)} films sélectionnés.")

In [ ]:
# On trie par note (averageRating) du plus grand au plus petit (ascending=False)
top_films = df_art_essai.sort_values(by='averageRating', ascending=False)

# On affiche le Top 10 avec juste les colonnes intéressantes
colonnes_a_voir = ['primaryTitle', 'startYear', 'averageRating', 'numVotes', 'genres']
top_films[colonnes_a_voir].head(15)

In [ ]:
# --- ÉTAPE 1 : Préparation ---
chemin_principals = "données/title.principals.tsv.gz"
colonnes_a_garder = ['tconst', 'nconst', 'category']

# On prépare la liste de tes films (on la transforme en "set" pour que la recherche soit instantanée)
# C'est une astuce d'optimisation très importante
ids_art_essai_set = set(df_art_essai['tconst'])

print("Démarrage du traitement par morceaux (Chunks)...")

# --- ÉTAPE 2 : La boucle (Le saucisson) ---
liste_morceaux_gardes = []
taille_morceau = 1000000  # On lit 1 million de lignes à la fois

# On ouvre le fichier en mode "lecture par morceaux"
lecteur = pd.read_csv(chemin_principals, sep='\t', compression='gzip', 
                      usecols=colonnes_a_garder, chunksize=taille_morceau)

# Pour chaque morceau du fichier :
for i, chunk in enumerate(lecteur):
    # 1. On ne garde que les acteurs/actrices
    chunk_acteurs = chunk[chunk['category'].isin(['actor', 'actress'])]
    
    # 2. On ne garde que ceux qui sont dans TES films
    chunk_filtre = chunk_acteurs[chunk_acteurs['tconst'].isin(ids_art_essai_set)]
    
    # 3. On met le résultat de côté
    liste_morceaux_gardes.append(chunk_filtre)
    
    # Petit message pour savoir où on en est (tous les 10 millions de lignes)
    if i % 10 == 0:
        print(f"Traitement du morceau n°{i}...")

# --- ÉTAPE 3 : Assemblage final ---
print("Assemblage final...")
df_casting_art_essai = pd.concat(liste_morceaux_gardes)

print(f"Terminé ! Nous avons identifié {len(df_casting_art_essai)} rôles.")
df_casting_art_essai.head()

In [ ]:
# --- ÉTAPE 1 : Préparation ---
chemin_names = "données/name.basics.tsv.gz"
# On veut l'identifiant, le nom, et l'année de naissance (pour ta question sur l'âge moyen)
colonnes_names = ['nconst', 'primaryName', 'birthYear']

# On prépare la liste des acteurs qu'on a trouvés à l'étape d'avant
# (Pour ne chercher que les noms de CES 736 433 personnes, pas les autres)
ids_acteurs_set = set(df_casting_art_essai['nconst'])

print("Recherche des noms des acteurs... (Technique du Saucisson activée)")

# --- ÉTAPE 2 : La boucle de lecture ---
liste_noms_gardes = []
taille_morceau = 1000000

lecteur_names = pd.read_csv(chemin_names, sep='\t', compression='gzip', 
                            usecols=colonnes_names, chunksize=taille_morceau)

for i, chunk in enumerate(lecteur_names):
    # On garde seulement les lignes qui correspondent à nos acteurs
    chunk_filtre = chunk[chunk['nconst'].isin(ids_acteurs_set)]
    
    liste_noms_gardes.append(chunk_filtre)
    
    if i % 10 == 0:
        print(f"Traitement du bloc n°{i}...")

# --- ÉTAPE 3 : Assemblage et Fusion Finale ---
print("Assemblage des noms...")
df_noms = pd.concat(liste_noms_gardes)

# LA DERNIÈRE FUSION (Merge) : On colle les noms à côté des rôles
# On fusionne df_casting_art_essai (les rôles) avec df_noms (les infos)
df_acteurs_complet = pd.merge(df_casting_art_essai, df_noms, on='nconst')

print(f"Tout est prêt ! Nous avons les noms pour {len(df_acteurs_complet)} rôles.")
df_acteurs_complet.head()

In [ ]:
# On compte combien de fois chaque nom apparaît
top_acteurs = df_acteurs_complet['primaryName'].value_counts().head(20)

# On affiche le résultat
print("=== TOP 20 DES ACTEURS (Films Art & Essai / Drame / Bio) ===")
print(top_acteurs)

#  graphique
top_acteurs.sort_values().plot(kind='barh', figsize=(10, 8), color='teal')
plt.title("Acteurs les plus présents dans ta sélection")
plt.xlabel("Nombre de films")
plt.grid(axis='x')
plt.show()

1. Le constat : L'hégémonie Indienne 🇮🇳
On voit tous ces noms (Anupam Kher, Mohanlal, Amitabh Bachchan...) ? Ce sont des légendes vivantes, mais en Inde.

Pourquoi ? Le cinéma indien produit énormément de films par an. Un acteur populaire là-bas peut tourner 10 à 15 films par an, alors qu'un acteur français ou américain en fera 1 ou 2.

Pour Tours : C'est une info cruciale. Si on se base uniquement sur le "nombre de films" pour recommander, on va transformer le cinéma Art et Essai en cinéma Bollywood ! (Ce qui est une niche, mais peut-être pas le but principal).

2. Les pépites pour le projet "Tours" 🇫🇷
Dans la deuxième image, il y a deux noms qui sauvent la mise pour la stratégie locale :

Gérard Depardieu (117 films) : C'est la preuve que ta base contient bien du cinéma français patrimonial. Il est noyé dans la masse.

John Wayne (118 films) : Le roi du Western et du film classique américain. Parfait pour des rétrospectives.

In [ ]:
# 1. On nettoie les années de naissance
# (Parfois il y a des "\N" ou des erreurs, on force la conversion en nombres)
df_acteurs_complet['birthYear'] = pd.to_numeric(df_acteurs_complet['birthYear'], errors='coerce')

# 2. On récupère l'année du film (startYear)
# Attention : df_acteurs_complet ne l'a pas encore, il faut aller la chercher dans df_movies
# On fait une petite fusion rapide pour récupérer juste la colonne 'startYear'
df_ages = pd.merge(df_acteurs_complet, df_movies[['tconst', 'startYear']], on='tconst')

# 3. Le calcul de l'âge : Année du film - Année de naissance
df_ages['age_au_tournage'] = df_ages['startYear'] - df_ages['birthYear']

# 4. On filtre les aberrations (les âges négatifs ou > 100 ans qui sont des erreurs de données)
df_ages = df_ages[(df_ages['age_au_tournage'] > 0) & (df_ages['age_au_tournage'] < 100)]

# 5. Résultats
age_moyen = df_ages['age_au_tournage'].mean()
print(f"L'âge moyen des acteurs dans ta sélection Art et Essai est de : {age_moyen:.1f} ans.")

# 6. Petit graphique de la distribution des âges
import matplotlib.pyplot as plt

df_ages['age_au_tournage'].hist(bins=50, color='purple', edgecolor='black', alpha=0.7)
plt.title("À quel âge les acteurs tournent-ils ?")
plt.xlabel("Âge")
plt.ylabel("Nombre de rôles")
plt.axvline(age_moyen, color='red', linestyle='dashed', linewidth=1, label=f'Moyenne: {age_moyen:.1f} ans')
plt.legend()
plt.show()

C'est un chiffre très cohérent et fascinant à analyser ! 40,9 ans, c'est l'âge de la maturité.

Voici les 3 raisons principales qui pourraient expliquer ce résultat, liées directement à tes filtres :

1. L'effet "Art et Essai" (Le filtre de Genre) 🎭
On a filtré sur Drame, Biographie, Histoire.

Dans un film d'action ou d'horreur (type "Scream"), les acteurs sont souvent très jeunes (20-25 ans).

Dans un Drame ou une Biographie, on raconte souvent des histoires de vie, de couples qui se déchirent, ou de figures historiques. Ces rôles demandent de la complexité et du "vécu". On confie rarement le rôle d'un président ou d'une mère de famille à un acteur de 20 ans.

2. Le temps de la réussite (Le filtre de Popularité) ⭐
On a gardé les films avec plus de 100 votes.

Pour être dans un film connu et bien noté, il faut souvent être un acteur confirmé.

Les acteurs commencent jeunes, mais ils tournent le maximum de leurs films une fois qu'ils sont installés dans le métier, souvent entre 30 et 50 ans. C'est le "ventre mou" de la carrière.

3. Le biais Hommes / Femmes (La triste réalité du cinéma) 🚻
C'est l'hypothèse la plus intéressante sociologiquement.

L'histoire du cinéma (surtout avant les années 2000) a tendance à faire jouer les hommes très vieux (pense à Sean Connery ou John Wayne qui jouaient les séducteurs à 60 ans passés).

À l'inverse, les carrières des femmes s'arrêtaient souvent plus tôt ou se raréfiaient passé 40 ans (c'est moins vrai aujourd'hui, mais ta base contient des vieux films).

Comme il y a généralement plus de rôles masculins dans la base historique, leur âge plus élevé "tire" la moyenne vers le haut (vers 41 ans).

In [ ]:
# On groupe par "category" (actor vs actress) et on demande la moyenne d'âge
comparaison_genres = df_ages.groupby('category')['age_au_tournage'].mean()

print(comparaison_genres)

# Graphique pour visualiser l'écart
comparaison_genres.plot(kind='bar', color=['pink', 'blue'], rot=0)
plt.title("Moyenne d'âge : Actrices vs Acteurs")
plt.ylabel("Âge moyen")
plt.show()

In [ ]:
# 1. A quoi ressemblent les genres actuellement
print("Avant encodage :")
print(df_art_essai['genres'].head(3))

# 2. La transformation magique (get_dummies)
# "Sépare les mots par la virgule, et fais-en des colonnes"
df_genres_binaires = df_art_essai['genres'].str.get_dummies(sep=',')

# 3. Regardons le résultat
print("\nAprès encodage (Tableau de 0 et 1) :")
df_genres_binaires.head()

In [ ]:
# 1. On installe le cerveau (scikit-learn)
%pip install scikit-learn

# 2. On importe l'outil KNN (K-Nearest Neighbors)
from sklearn.neighbors import NearestNeighbors

# 3. On configure le modèle
# metric='cosine' : C'est la meilleure méthode pour comparer des textes/genres
# n_neighbors=6 : On veut le film lui-même + les 5 recommandations les plus proches
modele_knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=6)

# 4. On entraîne ! (Fit)
modele_knn.fit(df_genres_binaires)

print("✅ Le modèle est entraîné et prêt à faire des recommandations !")

In [ ]:
def recommander_films(titre_film):
    # 1. On cherche si le film existe dans notre base
    # On ignore la casse (majuscule/minuscule) pour être sympa avec l'utilisateur
    films_trouves = df_art_essai[df_art_essai['primaryTitle'].str.contains(titre_film, case=False, na=False)]
    
    if len(films_trouves) == 0:
        return "! Film introuvable dans la base Art et Essai."
    
    # On prend le premier résultat trouvé (le plus probable)
    index_film = films_trouves.index[0]
    titre_complet = films_trouves.iloc[0]['primaryTitle']
    print(f"Film sélectionné : {titre_complet}")
    
    # 2. On demande au modèle les voisins (KNN)
    distances, indices = modele_knn.kneighbors(df_genres_binaires.loc[[index_film]])
    
    # 3. On affiche les résultats
    print("\n--- 🍿 Recommandations pour le public de Tours ---")
    
    # On boucle sur les voisins trouvés (on saute le 1er car c'est le film lui-même)
    for i in range(1, len(indices[0])):
        index_reco = indices[0][i]
        titre_reco = df_art_essai.iloc[index_reco]['primaryTitle']
        genre_reco = df_art_essai.iloc[index_reco]['genres']
        note_reco = df_art_essai.iloc[index_reco]['averageRating']
        
        print(f"{i}. {titre_reco} ({genre_reco}) - Note: {note_reco}/10")

# --- TEST ---
# Essaie avec un film classique pour voir !
recommander_films("Godfather")

In [ ]:
# Essaie avec "The Godfather" pour être plus précise
recommander_films("The Godfather")